In [1]:
import pandas as pd
from pylab import rcParams
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn import metrics 
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.formula.api as sm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,PowerTransformer,PolynomialFeatures
from sklearn.linear_model import LinearRegression
import operator
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from pandas import DataFrame
from sklearn.ensemble import RandomForestRegressor

# 失业率回归模型（美国）

In [2]:
df=pd.read_csv("/home/mw/input/Unemp6069/unemp_yearly.csv", header=0, index_col=0)
df_USA2 = df[df['LOCATION']=="USA"]
df_USA2.reset_index(drop=True, inplace=True)
df_USA2 = df_USA2[df_USA2.TIME != 2020]

In [3]:
rolling_mean = df_USA2["Value"].rolling(7).mean()
rolling_std = df_USA2["Value"].rolling(7).std()
plt.plot(df_USA2["TIME"],df_USA2["Value"], color = 'blue', label = 'Original')
plt.plot(df_USA2["TIME"],rolling_mean, color = 'red', label = 'Rolling Mean')
plt.plot(df_USA2["TIME"],rolling_std, color = 'black', label = 'Rolling Std')
plt.legend(loc = 'best')
plt.title('Rolling Mean & Rolling Standard Deviation')
plt.xlabel('Year', fontweight ='bold') 
plt.ylabel('Unemployment rate(%)', fontweight ='bold')
plt.show()

<Figure size 432x288 with 1 Axes>

In [4]:
result = adfuller(df_USA2["Value"])
print('ADF Statistic: {}'.format(result[0]))
print('p-value: {}'.format(result[1]))
print('Critical Values:')
for key, value in result[4].items():
    print('\t{}: {}'.format(key, value))

ADF Statistic: -4.2488841461935225
p-value: 0.0005440043525607224
Critical Values:
	1%: -4.223238279489106
	5%: -3.189368925619835
	10%: -2.729839421487603


In [5]:
sm.graphics.tsa.plot_acf(df_USA2["Value"].squeeze())
plt.show()

<Figure size 432x288 with 1 Axes>

# ARIMA 模型的应用

In [6]:
 model = ARIMA(df_USA2['Value'], order=(1,0,1), exog = df_USA2['TIME'])
results = model.fit(disp=0)
plt.plot(df_USA2['TIME'],df_USA2['Value'], label='Actual data')
plt.plot(df_USA2['TIME'],results.fittedvalues, color='red', label='Forecasted data')
plt.legend(loc = 'best')
plt.xlabel('Year', fontweight ='bold') 
plt.ylabel('Unemployment rate(%)', fontweight ='bold')
plt.title('Unemployment rate-America')

Text(0.5, 1.0, 'Unemployment rate-America')

<Figure size 432x288 with 1 Axes>

In [7]:
results.plot_predict(0,20,exog = df_USA2['TIME'])

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

In [8]:
results.predict(20,20,exog = df_USA2['TIME'])

20    4.012981
dtype: float64

**2020年美国失业率预测值**

# 失业率回归模型（俄罗斯）

In [9]:
df=pd.read_csv('/home/mw/input/Unemp6069/clean_unemployment.csv')

In [10]:
df=df[df['LOCATION']=='RUS']
plt.plot(df['TIME'],df['Value'])
plt.xlabel('Year', fontweight ='bold') 
plt.ylabel('Unemployment rate(%)', fontweight ='bold')
plt.title('Unemployment rate-Russia')

Text(0.5, 1.0, 'Unemployment rate-Russia')

<Figure size 432x288 with 1 Axes>

In [11]:
x=[i for i in df['TIME']]
y=[i for i in df['Value']]
x_array=x
y_array=y
x=np.asarray(x)
y=np.asarray(y)
x = x[:, np.newaxis]
y = y[:, np.newaxis]
l=int(len(x)*.8)
x_train,x_test=x[:l],x[l:]
y_train,y_test=y[:l],y[l:]

In [12]:
y_temp=[]
for i in y:
    y_temp.append(list(i))
y=y_temp
y.remove([5.56738874942334])
y.remove([5.53719568468794])

x=list(x)
x.remove([2016])
x.remove([2015])

# 三阶多项式回归模型

In [13]:
polynomial_features= PolynomialFeatures(degree=3)
x_poly = polynomial_features.fit_transform(x_train)
model = LinearRegression()
model.fit(x_poly, y_train)
x_test_poly=polynomial_features.fit_transform(x_test)
y_poly_pred = model.predict(x_test_poly)
rmse = np.sqrt(mean_squared_error(y_poly_pred,y_test))
r2 = r2_score(y_test,y_poly_pred)
print("RMSE=",rmse)
print("R方值=",r2)

RMSE= 4.5082411913695095
R方值= -64.97702405546536


In [14]:
polynomial_features= PolynomialFeatures(degree=3)
x_poly = polynomial_features.fit_transform(x)

model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly)
rmse = np.sqrt(mean_squared_error(y,y_poly_pred))
r2 = r2_score(y,y_poly_pred)
plt.scatter(x, y, s=10)
sort_axis = operator.itemgetter(0)
sorted_zip = sorted(zip(x,y_poly_pred), key=sort_axis)
x, y_poly_pred = zip(*sorted_zip)
plt.plot(x, y_poly_pred, color='m')
plt.legend(['Predicted data','Actual data'])
plt.xlabel('Year', fontweight ='bold') 
plt.ylabel('Unemployment rate(%)', fontweight ='bold')
plt.title('Unemployment rate-Russia')
plt.show()

<Figure size 432x288 with 1 Axes>

In [15]:
scaler = StandardScaler()
dataset={'x':np.asarray(x).reshape(1,len(x))[0],'y':np.asarray(y).reshape(1,len(y))[0]}
df_data=pd.DataFrame(dataset)
normalized = scaler.fit_transform(df_data)
inverse = scaler.inverse_transform(normalized)
tobescaled=[]
for i in range(len(x)):
    tobescaled.append([x[i][0],y[i][0]])
tobescaled=np.asarray(tobescaled)

In [16]:
scaled = scaler.fit_transform(tobescaled)
x_scaled=[]
for i in range(len(x)):
    if(i==9):
        pass
    else:
        x_scaled.append([scaled[i][0]])
y_scaled=[]
for i in range(len(y)):
    if(i==9):
        pass
    else:
        y_scaled.append(scaled[i][1])
l=int(len(x)*.8)
x_scaled=x_scaled[:-1]
y_scaled=y_scaled[:-1]
x_train,x_test=x_scaled[:l],x_scaled[l:]
y_train,y_test=y_scaled[:l],y_scaled[l:]

# 支持向量回归模型 (SVR)

In [17]:
regressor = SVR(kernel='rbf')
regressor.fit(x_scaled,y_scaled)
y_pred = regressor.predict(x_scaled)
plt.scatter(x_scaled, y_scaled, color = 'magenta')
plt.plot(x_scaled, regressor.predict(x_scaled), color = 'green')
plt.title('Unemployment rate-Russia')
plt.legend(['Predicted data','Actual data'])
plt.xlabel('Year', fontweight ='bold') 
plt.ylabel('Unemployment rate(%)', fontweight ='bold')
plt.show()
l=int(len(x)*.8)
x_train,x_test=x_scaled[:l],x_scaled[l:]
y_train,y_test=y_scaled[:l],y_scaled[l:]
y_svr_pred=regressor.predict(x_test)
rmse = np.sqrt(mean_squared_error(regressor.predict(x_train),y_train))
r2 = r2_score(y_train,regressor.predict(x_train))
print("RMSE=",rmse)
print("R方=",r2)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


<Figure size 432x288 with 1 Axes>

RMSE= 0.41974873623506476
R方= 0.8059404690233242


In [18]:
l=int(len(x)*.8)
x_train,x_test=x_scaled[:l],x_scaled[l:]
y_train,y_test=y_scaled[:l],y_scaled[l:]
y_svr_pred=regressor.predict(x_test)
rmse = np.sqrt(mean_squared_error(regressor.predict(x_test),y_test))
r2 = r2_score(y_test,regressor.predict(x_test))
print("RMSE=",rmse)
print("R方=",r2)
sc_x=StandardScaler()
sc_x.fit_transform(x)
sc_y=StandardScaler()
sc_y.fit_transform(y)
y_pred = sc_y.inverse_transform ((regressor.predict (sc_x.transform(np.array([[2020]])))))

RMSE= 0.20475969870503768
R方= -8.053564999763868


In [19]:
y_pred

array([5.1265164])

**2020年俄罗斯失业率预测值**

# 失业率回归模型（南非）

In [30]:
df=pd.read_csv('/home/mw/input/Unemp6069/clean_unemployment.csv')
df=df[df['LOCATION']=='ZAF']

In [31]:
x=[i for i in df['TIME']]
y=[i for i in df['Value']]
x_array=x
y_array=y
x=np.asarray(x)
y=np.asarray(y)
x = x[:, np.newaxis]
y = y[:, np.newaxis]
l=int(len(x)*.8)
x_train,x_test=x[:l],x[l:]
y_train,y_test=y[:l],y[l:]

In [32]:
from keras import models, layers
from sklearn.preprocessing import StandardScaler
from keras.regularizers import l2
scaler = StandardScaler()

In [33]:
dataset={'x':x.reshape(1,len(x))[0],'y':y.reshape(1,len(y))[0]}
df_data=pd.DataFrame(dataset)
tobescaled=[]
for i in range(len(x)):
    tobescaled.append([x[i][0],y[i][0]])
tobescaled=np.asarray(tobescaled)
scaler = StandardScaler()
scaled = scaler.fit_transform(tobescaled)
x_scaled=[]
for i in range(len(x)):
    x_scaled.append([scaled[i][0]])
y_scaled=[]
for i in range(len(y)):
    y_scaled.append(scaled[i][1])

# 人工神经网络(ANN)

In [34]:
model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=[x_train.shape[1]]))
model.add(layers.Dense(16,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
model.add(layers.Dense(1))

In [35]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
y_scaled_nn=np.asarray(y_scaled).reshape(y.shape)

In [36]:
history = model.fit(np.asarray(x_scaled[:-1]),np.asarray(y_scaled_nn[:-1]), validation_split=0.2, epochs=600)

Train on 15 samples, validate on 4 samples
Epoch 1/600
15/15 [==============================] - 0s 12ms/step - loss: 0.6245 - mean_absolute_error: 0.5772 - val_loss: 0.6568 - val_mean_absolute_error: 0.6926
Epoch 2/600
15/15 [==============================] - 0s 85us/step - loss: 0.6002 - mean_absolute_error: 0.5626 - val_loss: 0.6789 - val_mean_absolute_error: 0.7087
Epoch 3/600
15/15 [==============================] - 0s 79us/step - loss: 0.5835 - mean_absolute_error: 0.5528 - val_loss: 0.6936 - val_mean_absolute_error: 0.7193
Epoch 4/600
15/15 [==============================] - 0s 77us/step - loss: 0.5699 - mean_absolute_error: 0.5449 - val_loss: 0.7070 - val_mean_absolute_error: 0.7288
Epoch 5/600
15/15 [==============================] - 0s 77us/step - loss: 0.5588 - mean_absolute_error: 0.5381 - val_loss: 0.7181 - val_mean_absolute_error: 0.7366
Epoch 6/600
15/15 [==============================] - 0s 80us/step - loss: 0.5492 - mean_absolute_error: 0.5322 - val_loss: 0.7284 - val_m

In [37]:
pred_values=model.predict(np.array(x_scaled).reshape(-1,1))
plt.plot(x_scaled[:-1],pred_values[:-1],'b')
plt.plot(x_scaled[:-1],y_scaled[:-1],'r')
plt.title('Unemployment rate-South Africa')
plt.legend(labels=['Predicted data', 'Actual data'])
plt.xlabel('Year', fontweight ='bold') 
plt.ylabel('Unemployment rate(%)', fontweight ='bold')

Text(0, 0.5, 'Unemployment rate(%)')

<Figure size 432x288 with 1 Axes>

In [38]:
rmse = np.sqrt(mean_squared_error(pred_values[:-1],y_scaled[:-1]))
r2 = r2_score(y_scaled[:-1],pred_values[:-1])
print("RMSE=",rmse)
print("R方=",r2)

RMSE= 0.3870854288753569
R方= 0.680272149780746


In [39]:
sc_x=StandardScaler()
sc_x.fit_transform(x)
sc_y=StandardScaler()
sc_y.fit_transform(y)
y_pred = sc_y.inverse_transform ((model.predict (sc_x.transform(np.array([[2020]])))))

In [40]:
l=int(len(x)*.8)
x_train,x_test=x_scaled[:l],x_scaled[l:]
y_train,y_test=y_scaled[:l],y_scaled[l:]
y_svr_pred=regressor.predict(x_train)
rmse = np.sqrt(mean_squared_error(regressor.predict(x_train),y_train))
r2 = r2_score(y_train,regressor.predict(x_train))
print("RMSE=",rmse)
print("R方=",r2)

RMSE= 1.046877447621807
R方= -2.526624305584964


In [41]:
y_pred[0]

array([26.01967], dtype=float32)

**2020年南非失业率预测值**